In [2]:
import numpy as np
import pandas as pd
import os, sys
import warnings
from tqdm import tqdm

warnings.filterwarnings('ignore')


%cd C:\emobility

C:\emobility


In [3]:
replacement_bybike = pd.read_pickle(f"./dbs/output/replacement_bybike.pkl")
emission_bylink = pd.read_pickle(f"./dbs/output/emission_bylink.pkl")


In [4]:

replacement_bybike['changed_act_id'] = replacement_bybike.groupby('person').cumcount()


In [5]:
emission_bylink = pd.merge(emission_bylink, replacement_bybike[["person",'changed_act_id','tour_replacement']], left_on= ["person",'act_id'], right_on= ["person",'changed_act_id'], how="left")

In [93]:
#emission_bylink[pd.isnull(emission_bylink['tour_replacement'])]
emission_bylink_bike = emission_bylink[emission_bylink.tour_replacement==True]
print('the reduction capacity: ', emission_bylink_bike['link_emission'].sum())

the reduction capacity:  607131.1219322365


In [94]:
emission_bylink_car = emission_bylink[emission_bylink.tour_replacement==False]
print('the reduced emission: ', emission_bylink_car['link_emission'].sum())

the reduced emission:  1904626.2304087048


In [109]:
emission_bylink['link_emission'].sum()

2511757.352340963

In [16]:
emission_bylink.tour_replacement.value_counts()

False    41052758
True     27586844
Name: tour_replacement, dtype: int64

In [28]:
emission_bylink['link_emission']= emission_bylink['link_emission'].div(emission_bylink['num_Deso'])

In [29]:
emission_bylink= emission_bylink.loc[emission_bylink.index.repeat(emission_bylink.num_Deso)]

In [30]:
emission_bylink['num_Deso']= emission_bylink.groupby(['person','seq']).cumcount()

In [25]:
def link_todeso(row):
    if len(row['Deso']) == 1:
        Deso = row['Deso'][0]
    else:
        Deso = row['Deso'][row['num_Deso']]
    return Deso

In [31]:
emission_bylink['Deso'] = emission_bylink.apply( lambda row: link_todeso(row), axis=1)

In [32]:
emission_bydeso = emission_bylink.groupby(['Deso'])['link_emission'].sum().reset_index()

### Visualization

In [127]:
import geopandas as gpd



DeSO = gpd.read_file(f'./dbs/emission/deso_statistik_shp/Fordon_region.shp')
DeSO = DeSO[['Deso','geometry']]

In [128]:
gdf=DeSO.merge(emission_bydeso,on="Deso")
gdf= gdf[gdf['Deso'].str[:2]=='14']


In [131]:
print('the total emission within VG: ', gdf.link_emission.sum())


the total emission within VG:  2182322.053552644


In [132]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable


fig, ax = plt.subplots(1, 1, figsize=(12, 12), dpi=250)

divider = make_axes_locatable(ax)
cax = divider.append_axes("bottom", size="5%", pad=0.1)

gdf.plot(column='link_emission',
         ax=ax,
         cmap='viridis',
         legend=True,
         cax=cax,
         legend_kwds={'label': "Greenhouse gas emissions from passenger cars by the driven DeSO zones (in kg CO2 eq.)", 'orientation': "horizontal"})

#cax.tick_params(labelsize='12')

#plt.show()
plt.savefig(r'.\figures\emission_abs.jpg')

In [98]:
emission_bylink_reduction = emission_bylink[emission_bylink.tour_replacement==True]
emission_bylink_reduction = emission_bylink_reduction.groupby(['Deso'])['link_emission'].sum().reset_index()



In [99]:
gdf_reduction = DeSO.merge(emission_bylink_reduction, on="Deso")
gdf_reduction = gdf_reduction[gdf_reduction['Deso'].str[:2] == '14']


fig, ax = plt.subplots(1, 1, figsize=(12, 12), dpi=250)

divider = make_axes_locatable(ax)
cax = divider.append_axes("bottom", size="5%", pad=0.1)

gdf_reduction.plot(column='link_emission',
         ax=ax,
         cmap='viridis',
         legend=True,
         cax=cax,
         legend_kwds={'label': "Reduction in greenhouse gas emissions from passenger cars by the driven DeSO zones (in kg CO2 eq.)", 'orientation': "horizontal"})

#plt.show()
plt.savefig(r'.\figures\emission_reduction_abs.jpg')


In [100]:
gdf_percentage_reduction = gdf.merge(gdf_reduction[['Deso','link_emission']], on="Deso")
gdf_percentage_reduction['percentage'] = (gdf_percentage_reduction['link_emission_y']/gdf_percentage_reduction['link_emission_x'])*100


fig, ax = plt.subplots(1, 1, figsize=(12, 12), dpi=250)

divider = make_axes_locatable(ax)
cax = divider.append_axes("bottom", size="5%", pad=0.1)

gdf_percentage_reduction.plot(column='percentage',
         ax=ax,
         cmap='viridis',
         legend=True,
         cax=cax,
         legend_kwds={'label': "Reduction in greenhouse gas emissions from passenger cars by the driven DeSO zones (in %)", 'orientation': "horizontal"})

#plt.show()
plt.savefig(r'.\figures\emission_reduction_per.jpg')



In [101]:
emission_byperson = emission_bylink.groupby(['person'])['link_emission'].sum().reset_index()


emission_byperson_reduction = emission_bylink[emission_bylink.tour_replacement==True]
emission_byperson_reduction = emission_byperson_reduction.groupby(['person'])['link_emission'].sum().reset_index()


In [102]:
df_act = pd.read_csv(f'./dbs/agents/1_activity_plans.csv')
df_act = df_act[df_act.act_purpose=='home']
df_act.drop_duplicates(subset=['person'], inplace=True)


In [103]:
emission_byperson = pd.merge(emission_byperson, df_act[['person','deso']], on='person', how="left")

emission_byperson_reduction = pd.merge(emission_byperson_reduction, df_act[['person','deso']], on='person', how="left")

In [104]:
emission_byresidentialzone = emission_byperson.groupby(['deso'])['link_emission'].sum().reset_index()

emission_byresidentialzone_reduction = emission_byperson_reduction.groupby(['deso'])['link_emission'].sum().reset_index()

In [105]:
gdf_residentialzone = gdf[['Deso','geometry']].merge(emission_byresidentialzone[['deso','link_emission']], left_on="Deso", right_on="deso")
gdf_residentialzone = gdf_residentialzone.merge(emission_byresidentialzone_reduction[['deso','link_emission']], left_on="Deso", right_on="deso")


In [106]:

gdf_residentialzone['percentage'] = (gdf_residentialzone['link_emission_y']/gdf_residentialzone['link_emission_x'])*100


fig, ax = plt.subplots(1, 1, figsize=(12, 12), dpi=250)

divider = make_axes_locatable(ax)
cax = divider.append_axes("bottom", size="5%", pad=0.1)

gdf_residentialzone.plot(column='percentage',
         ax=ax,
         cmap='viridis',
         legend=True,
         cax=cax,
         legend_kwds={'label': "Reduction in greenhouse gas emissions from passenger cars by the agents' residential DeSO zones (in %)", 'orientation': "horizontal"})

#plt.show()
plt.savefig(r'.\figures\emission_reduction_per_residential.jpg')

